In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate();

using Flux
using Flux: crossentropy, throttle
using Flux.Data: Tree, children, isleaf

include("data.jl")

N = 300

embedding = param(randn(N, length(alphabet)))

W = Dense(2N, N, tanh)
combine(a, b) = W([a; b])

sentiment = Chain(Dense(N, 5), softmax)

function forward(tree)
  if isleaf(tree)
    token, sent = tree.value
    phrase = embedding * token
    phrase, crossentropy(sentiment(phrase), sent)
  else
    _, sent = tree.value
    c1, l1 = forward(tree[1])
    c2, l2 = forward(tree[2])
    phrase = combine(c1, c2)
    phrase, l1 + l2 + crossentropy(sentiment(phrase), sent)
  end
end

loss(tree) = forward(tree)[2]

opt = ADAM(params(embedding, W, sentiment))
evalcb = () -> @show loss(train[1])

Flux.train!(loss, zip(train), opt,
            cb = throttle(evalcb, 10))

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Cloning git-repo `https://github.com/FluxML/Flux.jl.git`
  Updating git-repo `https://github.com/FluxML/Flux.jl.git`
┌ Info: Downloading sentiment treebank dataset
└ @ Flux.Data.Sentiment /home/dhairyagandhi96/.julia/packages/Flux/OLvp4/src/data/sentiment.jl:8
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   467  100   467    0     0     66      0  0:00:07  0:00:07 --:--:--   103
100  771k  100  771k    0     0  52053      0  0:00:15  0:00:15 --:--:--  213k
loss(train[1]) = 114.6365445661485 (tracked)
loss(train[1]) = 70.40135957767315 (tracked)
loss(train[1]) = 64.75842161996924 (tracked)
loss(train[1]) = 63.462926505281665 (tracked)
loss(train[1]) = 65.14176404205674 (tracked)
loss(train[1]) = 57.920682028834186 (tracked)
loss(train[1]) = 56.760135